<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/Copy_of_Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way.

All of those work in the same way: they add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.



## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

In [31]:
!pip install -q transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



In [34]:
import os, sys, shutil

os.chdir("/content")
current_dir = os.getcwd()
print(f"The current directory is {current_dir}")

if os.path.isdir('ML_GitHub'):
    shutil.rmtree('ML_GitHub')

!git init
!git branch -m main
!git clone https://github.com/claudelepere/ML_GitHub.git
print(f"list /content: {os.listdir(current_dir)}")
print(f"list /content/ML_GitHub: {os.listdir(current_dir + '/ML_GitHub')}")
!ls -la /content/ML_GitHub/datasetHF
os.chdir(current_dir + '/ML_GitHub')


!pip install -q transformers datasets
from datasets import DatasetDict

dataset = DatasetDict.load_from_disk('datasetHF')
dataset



The current directory is /content
Reinitialized existing Git repository in /content/.git/
Cloning into 'ML_GitHub'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 56 (delta 25), reused 50 (delta 22), pack-reused 0 (from 0)
Receiving objects: 100% (56/56), 9.93 MiB | 13.56 MiB/s, done.
Resolving deltas: 100% (25/25), done.
list /content: ['.config', 'ML_GitHub', '.git', 'sample_data']
list /content/ML_GitHub: ['README.md', 'datasetHF', '.git']
total 24
drwxr-xr-x 5 root root 4096 Nov  4 22:59 .
drwxr-xr-x 4 root root 4096 Nov  4 22:59 ..
-rw-r--r-- 1 root root   43 Nov  4 22:59 dataset_dict.json
drwxr-xr-x 2 root root 4096 Nov  4 22:59 test
drwxr-xr-x 2 root root 4096 Nov  4 22:59 train
drwxr-xr-x 2 root root 4096 Nov  4 22:59 validation


DatasetDict({
    train: Dataset({
        features: ['id', 'text', '394', '142', '146', '147', '148', '149', '150', '151', '408', '409', '153', '154', '155', '156', '157', '158', '160', '152', '162', '165', '167', '168', '169', '170', '171', '685', '174', '686', '176', '689', '173', '356', '360', '361', '362', '364', '760', '756', '758', '375', '376', '761'],
        num_rows: 128
    })
    validation: Dataset({
        features: ['id', 'text', '394', '142', '146', '147', '148', '149', '150', '151', '408', '409', '153', '154', '155', '156', '157', '158', '160', '152', '162', '165', '167', '168', '169', '170', '171', '685', '174', '686', '176', '689', '173', '356', '360', '361', '362', '364', '760', '756', '758', '375', '376', '761'],
        num_rows: 18
    })
    test: Dataset({
        features: ['id', 'text', '394', '142', '146', '147', '148', '149', '150', '151', '408', '409', '153', '154', '155', '156', '157', '158', '160', '152', '162', '165', '167', '168', '169', '170', '171'

As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', '394', '142', '143', '146', '147', '148', '149', '150', '151', '408', '409', '153', '154', '155', '156', '157', '158', '160', '152', '162', '667', '165', '167', '168', '169', '170', '171', '685', '174', '686', '176', '689', '173', '175', '356', '360', '361', '362', '364', '760', '371', '756', '373', '758', '375', '376', '761', '757'],
        num_rows: 640
    })
    validation: Dataset({
        features: ['id', 'text', '394', '142', '143', '146', '147', '148', '149', '150', '151', '408', '409', '153', '154', '155', '156', '157', '158', '160', '152', '162', '667', '165', '167', '168', '169', '170', '171', '685', '174', '686', '176', '689', '173', '175', '356', '360', '361', '362', '364', '760', '371', '756', '373', '758', '375', '376', '761', '757'],
        num_rows: 90
    })
    test: Dataset({
        features: ['id', 'text', '394', '142', '143', '146', '147', '148', '149', '150', '151', '408', '409', '153', '154'

Let's check the first example of the training split:

In [ ]:
example = dataset['train'][0]
example

{'id': 167804,
 'text': "Dilaco - Fiber Welding Lassen, Welding Dilaco Infinite reasons to choose for Dilaco Do you want to boost your career? Are you eager to learn and do you want to attain success ? If so, you're at the right address. At Dilaco, together, we built your future ! For our new project we're looking for you. You're someone who has an hands-on mentality, you like being on the road, but especially you deliver high quality work. A higher degree is not essential. We offer you a training to become a certified fiber welder ! Job related skills You're fluent in Dutch You have a driver's license You're responsible for welding fiber cables You're responsible for the assembling and installation in order to correctly connect the customer's residence to the fiber network. You're able to read and understand a technical plan Personal skills Reliable Patient Customer oriented Hands-on mentality Our offer After a short welding experience, we offer you the possibility to develop yourself

The dataset consists of tweets, labeled with one or more emotions.

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [ ]:
labels = [label for label in dataset['train'].features.keys() if label not in ['id', 'text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['394',
 '142',
 '143',
 '146',
 '147',
 '148',
 '149',
 '150',
 '151',
 '408',
 '409',
 '153',
 '154',
 '155',
 '156',
 '157',
 '158',
 '160',
 '152',
 '162',
 '667',
 '165',
 '167',
 '168',
 '169',
 '170',
 '171',
 '685',
 '174',
 '686',
 '176',
 '689',
 '173',
 '175',
 '356',
 '360',
 '361',
 '362',
 '364',
 '760',
 '371',
 '756',
 '373',
 '758',
 '375',
 '376',
 '761',
 '757']

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/640 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/270 [00:00<?, ? examples/s]

In [ ]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [ ]:
tokenizer.decode(example['input_ids'])

"[CLS] dilaco - fiber welding lassen, welding dilaco infinite reasons to choose for dilaco do you want to boost your career? are you eager to learn and do you want to attain success? if so, you're at the right address. at dilaco, together, we built your future! for our new project we're looking for you. you're someone who has an hands - on mentality, you like being on the road, but especially you deliver high quality work. a higher degree is not essential. we offer you a training to become a certified fiber welder! job related skills you're [SEP]"

In [ ]:
example['labels']

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [ ]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['156', '157', '158', '160', '685']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html).

In [ ]:
encoded_dataset.set_format("torch")

## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things:

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [ ]:
batch_size = 8
metric_name = "f1"

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [ ]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [ ]:
encoded_dataset['train']['input_ids'][0]

tensor([  101, 29454, 22684,  1011, 11917, 23604, 27333,  2368,  1010, 23604,
        29454, 22684, 10709,  4436,  2000,  5454,  2005, 29454, 22684,  2079,
         2017,  2215,  2000, 12992,  2115,  2476,  1029,  2024,  2017,  9461,
         2000,  4553,  1998,  2079,  2017,  2215,  2000, 18759,  3112,  1029,
         2065,  2061,  1010,  2017,  1005,  2128,  2012,  1996,  2157,  4769,
         1012,  2012, 29454, 22684,  1010,  2362,  1010,  2057,  2328,  2115,
         2925,   999,  2005,  2256,  2047,  2622,  2057,  1005,  2128,  2559,
         2005,  2017,  1012,  2017,  1005,  2128,  2619,  2040,  2038,  2019,
         2398,  1011,  2006,  5177,  3012,  1010,  2017,  2066,  2108,  2006,
         1996,  2346,  1010,  2021,  2926,  2017,  8116,  2152,  3737,  2147,
         1012,  1037,  3020,  3014,  2003,  2025,  6827,  1012,  2057,  3749,
         2017,  1037,  2731,  2000,  2468,  1037,  7378, 11917,  2057, 16502,
          999,  3105,  3141,  4813,  2017,  1005,  2128,   102])

In [ ]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7214, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.1897,  0.5491, -0.2574, -0.0448, -0.1342, -0.7379,  0.2182,  0.0864,
          0.2418, -0.0774,  0.3886, -0.5678, -0.0662, -0.1910,  0.0348,  0.2666,
         -0.4750, -0.1375,  0.0427,  0.0346, -0.0508,  0.0691,  0.3466,  0.5626,
         -0.0015,  0.3688,  0.4131, -0.0221, -0.0441, -0.4900,  0.0597,  0.6614,
          0.1142, -0.9726, -0.2180,  0.3647,  0.1889,  0.0477, -0.2885, -0.0838,
          0.0149, -0.2674,  0.3377,  0.4434,  0.0642,  0.4184, -0.0641, -0.2028]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

Let's start training!

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.281728,0.000000,0.500000,0.000000
2,No log,0.198168,0.000000,0.500000,0.000000
3,No log,0.175567,0.000000,0.500000,0.000000
4,No log,0.167872,0.000000,0.500000,0.000000
5,No log,0.165980,0.000000,0.500000,0.000000


TrainOutput(global_step=400, training_loss=0.23786224365234376, metrics={'train_runtime': 3966.1718, 'train_samples_per_second': 0.807, 'train_steps_per_second': 0.101, 'total_flos': 210575779430400.0, 'train_loss': 0.23786224365234376, 'epoch': 5.0})

## Evaluate

After training, we evaluate our model on the validation set.

In [ ]:
trainer.evaluate()

{'eval_loss': 0.2817283570766449,
 'eval_f1': 0.0,
 'eval_roc_auc': 0.5,
 'eval_accuracy': 0.0,
 'eval_runtime': 27.1522,
 'eval_samples_per_second': 3.315,
 'eval_steps_per_second': 0.442,
 'epoch': 5.0}

## Inference

Let's test the model on a new sentence:

In [ ]:
text = "I'm happy I can finally train a model for multi-label classification"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [ ]:
logits = outputs.logits
logits.shape

torch.Size([1, 48])

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [ ]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

[]
